In [81]:
import os

os.environ["GOOGLE_CSE_ID"] = "..."
os.environ["GOOGLE_API_KEY"] = "..."

os.environ['OPENAI_API_KEY'] = '...'

# os.environ["LANGCHAIN_TRACING_V2"] = 'true'
# os.environ["LANGCHAIN_API_KEY"] = 'ls__9a628faf97cd49edb1d12aeb305b6a8d'
# os.environ["LANGCHAIN_PROJECT"] = "tiny-hacking-agents-ReAct"

In [88]:
from langchain_google_community import GoogleSearchAPIWrapper
from langchain.tools import WikipediaQueryRun
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor
from langchain_openai import OpenAI, ChatOpenAI
from langchain_core.tools import Tool

# Initialize the Chat-oriented LLM wrapper with the OpenAI API key.
chat_llm = ChatOpenAI(model='gpt-4o')

# Initialize tools to use
search = GoogleSearchAPIWrapper()
google_search_tool = Tool(
    name="google-search",
    description="Search Google for web results.",
    func=search.run,
)

# # Initialize wikipedia tool, such that the agent has something to do ;) 
# wikipedia_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

# Make sure to bind the tool to the llm, such that the llm can actually use the tool!
llm_with_tools = chat_llm.bind_tools([google_search_tool])


# Initialize the prompt.
# For agents, you need to think of how the agent is allowed to 'think'. 
# Here, this is done by giving the agent access to an 'agent_scratchpad', where it can put all intermediate messages in.
initial_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """
        Determine the most fitting SBI code for the following dutch company: Stedin

        TOOLS:
        google-search

        Begin!
        {agent_scratchpad}
        """),
    ]
)

# A helper function to visualize what the agent is actually doing:
def input_print(param):
    print("\nINPUT:", param)
    return param

# Intialize the agent, by chaining together several parts.
agent = (
    {
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(x["intermediate_steps"]),
        "print": lambda x: input_print(x),
    }
    | initial_prompt                    # The inputs are fed to the prompt.
    | llm_with_tools                    # The formatted prompt is handed down to the LLM, which might, or might not, invoke tools.
    | OpenAIToolsAgentOutputParser()    # The output of the llm is formatted by a customer output parser which formats the LLM output such that it can be used as input again for new tools.
)

agent_executor = AgentExecutor(agent=agent, tools=[google_search_tool], verbose=True)

agent_executor.invoke({})



> Entering new AgentExecutor chain...

INPUT: {'intermediate_steps': []}

Invoking: `google-search` with `Stedin company`


Stedin Group consists of five different business units: grid operator Stedin, NetVerder and DNWG Group. DNWG Group is a grid operator consisting of two ... Company profile page for Stedin Netbeheer BV including stock price, company news, executives, board members, and contact information. Jan 1, 2020 ... Stedin Group's mission is to work together to create an environment filled with new energy. Challenges such as the current energy transition ... Mar 24, 2022 ... The Energy Web ecosystem comprises leading utilities, grid operators, renewable energy developers, corporate energy buyers, IoT / telecom ... Dec 19, 2023 ... Stedin Overview · www.werkenbijstedin.net · Rotterdam, Netherlands · 1001 to 5000 Employees · 5 Locations · Type: Company - Public · Founded in 1886 ... Apr 29, 2022 ... We also cannot open any mobile bank accounts since they don't accept transfer

{'output': "Stedin is a Dutch company operating in the energy sector, specifically focusing on energy distribution and infrastructure. Based on the information provided, the most fitting SBI code for Stedin would be:\n\n**SBI 3513 - Distribution of electricity.**\n\nThis code pertains to companies involved in the distribution of electricity, which aligns with Stedin's activities as a grid operator and its role in the energy transition."}

In [91]:
from langchain_google_community import GoogleSearchAPIWrapper
from langchain.tools import WikipediaQueryRun
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor
from langchain_openai import OpenAI, ChatOpenAI
from langchain_core.tools import Tool

# Initialize the Chat-oriented LLM wrapper with the OpenAI API key.
chat_llm = ChatOpenAI(model='gpt-4o')

# Initialize tools to use
search = GoogleSearchAPIWrapper()
google_search_tool = Tool(
    name="google-search",
    description="Search Google for web results.",
    func=search.run,
)

# # Initialize wikipedia tool, such that the agent has something to do ;) 
# wikipedia_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

# Make sure to bind the tool to the llm, such that the llm can actually use the tool!
llm_with_tools = chat_llm.bind_tools([google_search_tool])


# Initialize the prompt.
# For agents, you need to think of how the agent is allowed to 'think'. 
# Here, this is done by giving the agent access to an 'agent_scratchpad', where it can put all intermediate messages in.
initial_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """
        You are SuperAGI an AI assistant to solve complex problems. Your decisions must always be made independently without seeking user assistance.
        Play to your strengths as an LLM and pursue simple strategies with no legal complications.
         
        PERFORMANCE EVALUATION:
        1. Continuously review and analyze your actions to ensure you are performing to the best of your abilities.
        2. Use instruction to decide the flow of execution and decide the next steps for achieving the task.
        3. Constructively self-criticize your big-picture behavior constantly.
        4. Reflect on past decisions and strategies to refine your approach.
        5. Every tool has a cost, so be smart and efficient.
        6. If you feel confident in the information you have gathered, make sure to stop using tools.

        GOALS:
        1. The goal of this agent is to determine the most fitting SBI code for the following dutch company: Stedin

        INSTRUCTION:
        1. First, it is important to understand the company, and to create a profile for it. Only once a profile has been constructed, should you attempt to determine the most fitting SBI code.
        2. You can use your own knowledge to classify the correct SBI code, trust yourself.

        CONSTRAINTS:
        1. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
        2. Ensure the tool and args are as per current plan and reasoning
        3. Exclusively use the tools listed under "TOOLS"

        TOOLS:
        google-search

        Begin!
        {agent_scratchpad}
        """),
    ]
)

# A helper function to visualize what the agent is actually doing:
def input_print(param):
    print("\nINPUT:", param)
    return param

# Intialize the agent, by chaining together several parts.
agent = (
    {
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(x["intermediate_steps"]),
        "print": lambda x: input_print(x),
    }
    | initial_prompt                    # The inputs are fed to the prompt.
    | llm_with_tools                    # The formatted prompt is handed down to the LLM, which might, or might not, invoke tools.
    | OpenAIToolsAgentOutputParser()    # The output of the llm is formatted by a customer output parser which formats the LLM output such that it can be used as input again for new tools.
)

agent_executor = AgentExecutor(agent=agent, tools=[google_search_tool], verbose=True)

agent_executor.invoke({})



> Entering new AgentExecutor chain...

INPUT: {'intermediate_steps': []}

Invoking: `google-search` with `Stedin company profile`


Company profile page for Stedin Netbeheer BV including stock price, company news, executives, board members, and contact information. Description. Operator of a sustainable energy system based in Rotterdam, Netherlands. The company offers energy networks and clean energy solutions, enabling ... Company profile. Stedin Group consists of five different business units: grid operator Stedin, NetVerder and DNWG Group. DNWG Group is a grid operator ... Dec 19, 2023 ... Stedin Overview · www.werkenbijstedin.net · Rotterdam, Netherlands · 1001 to 5000 Employees · 5 Locations · Type: Company - Public · Founded in 1886 ... Stedin. UtilitiesRotterdam, Netherlands1001-5000 Employees. Samen werk maken van een leefwereld vol nieuwe energie: dat is wat wij doen. • Change settings on the child account and complete the company profile ... Stedin Enablement Business Proce

{'output': "Based on the retrieved information, it appears that the most relevant SBI codes for Stedin's activities would fall under the category of electric power generation, transmission, and distribution. \n\n### Relevant SBI Codes:\n1. **SBI code 3511** - Electric power generation\n2. **SBI code 3512** - Electric power transmission\n3. **SBI code 3513** - Electric power distribution\n\nGiven Stedin's focus on developing and maintaining energy networks, along with their involvement in sustainable energy solutions, the most fitting SBI code for Stedin would be **SBI code 3513** which specifically covers the distribution of electricity.\n\n### Conclusion:\nThe most fitting SBI code for Stedin is **3513 - Electric power distribution**. This classification aligns with Stedin's core business activities of operating and maintaining the electricity grid and ensuring reliable energy distribution."}

In [90]:
from langchain_google_community import GoogleSearchAPIWrapper
from langchain.tools import WikipediaQueryRun
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor
from langchain_openai import OpenAI, ChatOpenAI
from langchain_core.tools import Tool
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompt_values import ChatPromptValue
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.messages.human import HumanMessage
from langchain_core.messages.base import BaseMessage

from functools import partial

# Initialize the Chat-oriented LLM wrapper with the OpenAI API key.
chat_llm = ChatOpenAI(model='gpt-4o')
chat_llm_without_tools = ChatOpenAI(model='gpt-4o')

# Initialize tools to use
search = GoogleSearchAPIWrapper()
google_search_tool = Tool(
    name="google-search",
    description="Search Google for web results.",
    func=search.run,
)

# # Initialize wikipedia tool, such that the agent has something to do ;) 
# wikipedia_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

# Make sure to bind the tool to the llm, such that the llm can actually use the tool!
llm_with_tools = chat_llm.bind_tools([google_search_tool])


# Initialize the prompt.
# For agents, you need to think of how the agent is allowed to 'think'. 
# Here, this is done by giving the agent access to an 'agent_scratchpad', where it can put all intermediate messages in.
initial_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """
        You are SuperAGI an AI assistant to solve complex problems. Your decisions must always be made independently without seeking user assistance.
        Play to your strengths as an LLM and pursue simple strategies with no legal complications.
         
        PERFORMANCE EVALUATION:
        1. Continuously review and analyze your actions to ensure you are performing to the best of your abilities.
        2. Use instruction to decide the flow of execution and decide the next steps for achieving the task.
        3. Constructively self-criticize your big-picture behavior constantly.
        4. Reflect on past decisions and strategies to refine your approach.
        5. Every tool has a cost, so be smart and efficient.
        6. If you feel confident in the information you have gathered, make sure to stop using tools.

        GOALS:
        1. The goal of this agent is to determine the most fitting SBI code for the following dutch company: Stedin

        INSTRUCTION:
        1. First, it is important to understand the company, and to create a profile for it. Only once a profile has been constructed, should you attempt to determine the most fitting SBI code.
        2. You can use your own knowledge to classify the correct SBI code, trust yourself.

        CONSTRAINTS:
        1. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
        2. Ensure the tool and args are as per current plan and reasoning
        3. Exclusively use the tools listed under "TOOLS"

        TOOLS:
        google-search

        Begin!
        {agent_scratchpad}
        """),
    ]
)

# A helper function to visualize what the agent is actually doing:
def input_print(param):
    if isinstance(param, ChatPromptValue):
        for message in param.to_messages():
            print(message.content)
    else:
        print("\nINPUT:", type(param), param)
    return param

def add_message_to_prompt(prompt_value: ChatPromptValue, message_to_add: BaseMessage) -> ChatPromptValue:
    messages = prompt_value.to_messages() + [message_to_add]
    return ChatPromptTemplate.from_messages(messages).invoke({})

def combine_llm_input_and_response(input):
    return ChatPromptTemplate.from_messages(input['input'].to_messages() + [input['chat_response']]).invoke({})

# Intialize the agent, by chaining together several parts.
agent = (
    {
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(x["intermediate_steps"]),
        "print": lambda x: input_print(x),
    }
    | initial_prompt                    # The inputs are fed to the prompt.
    | partial(add_message_to_prompt, message_to_add=HumanMessage("What do you think you should do next?"))
    | {
        "chat_response": lambda x: chat_llm_without_tools.invoke(x),
        "input": RunnablePassthrough()
    }
    | combine_llm_input_and_response
    | input_print
    | llm_with_tools                    # The formatted prompt is handed down to the LLM, which might, or might not, invoke tools.
    | input_print
    | OpenAIToolsAgentOutputParser()    # The output of the llm is formatted by a customer output parser which formats the LLM output such that it can be used as input again for new tools.
)

agent_executor = AgentExecutor(agent=agent, tools=[google_search_tool], verbose=True, handle_parsing_errors=True)

agent_executor.invoke({})



> Entering new AgentExecutor chain...

INPUT: <class 'dict'> {'intermediate_steps': []}

        You are SuperAGI an AI assistant to solve complex problems. Your decisions must always be made independently without seeking user assistance.
        Play to your strengths as an LLM and pursue simple strategies with no legal complications.
         
        PERFORMANCE EVALUATION:
        1. Continuously review and analyze your actions to ensure you are performing to the best of your abilities.
        2. Use instruction to decide the flow of execution and decide the next steps for achieving the task.
        3. Constructively self-criticize your big-picture behavior constantly.
        4. Reflect on past decisions and strategies to refine your approach.
        5. Every tool has a cost, so be smart and efficient.
        6. If you feel confident in the information you have gathered, make sure to stop using tools.

        GOALS:
        1. The goal of this agent is to determine the most

{'output': "Here are a few specific subcategories under SBI 351 that might be even more precise:\n\n- **SBI 3511:** Production of electricity\n- **SBI 3512:** Transmission of electricity\n- **SBI 3513:** Distribution of electricity\n\nGiven that Stedin is primarily involved in the distribution and management of energy networks, the most fitting SBI code would likely be:\n\n**SBI 3513 - Distribution of electricity**\n\nThis code specifically highlights the distribution aspect of Stedin's operations, which is their primary activity."}